In [35]:
import numpy as np
from matplotlib import pyplot as plt
import math

img = np.arange(0,255,4).reshape((8, 8))
print(img)

[[  0   4   8  12  16  20  24  28]
 [ 32  36  40  44  48  52  56  60]
 [ 64  68  72  76  80  84  88  92]
 [ 96 100 104 108 112 116 120 124]
 [128 132 136 140 144 148 152 156]
 [160 164 168 172 176 180 184 188]
 [192 196 200 204 208 212 216 220]
 [224 228 232 236 240 244 248 252]]


In [51]:
rand_img = np.random.randint(0, 255, size = (9,9))
print(rand_img)

[[ 13  30  23 166 188 237 193 125 186]
 [126 190 161 180  29  67  59  41 169]
 [206 225  77  80  40  99 194  46 122]
 [140   1 103  64  58  14 186 226   1]
 [145 219 150 118 142 237 137  88  68]
 [ 78 108   2  40 242  40   3 201  34]
 [ 77  81 212  18 198  81  47 245 154]
 [245 184 174  59 175 166  43 166  50]
 [ 66 107  22  57  76 159 151 143  29]]


In [53]:
correlation_mtx(rand_img)

array([[ 1.        ,  0.63173596,  0.46808527, -0.15766216, -0.32463463,
        -0.10962613, -0.07380689, -0.24736541, -0.28958441],
       [ 0.63173596,  1.        ,  0.31675629,  0.17291587, -0.23904206,
         0.23450588, -0.20152667, -0.70220303,  0.11290152],
       [ 0.46808527,  0.31675629,  1.        , -0.04750146, -0.0824894 ,
        -0.04919215, -0.32220206,  0.05188949,  0.23458364],
       [-0.15766216,  0.17291587, -0.04750146,  1.        , -0.33503551,
         0.39685792,  0.30920178, -0.72716904,  0.56253581],
       [-0.32463463, -0.23904206, -0.0824894 , -0.33503551,  1.        ,
         0.22408705, -0.51717961,  0.5533037 ,  0.01013573],
       [-0.10962613,  0.23450588, -0.04919215,  0.39685792,  0.22408705,
         1.        ,  0.31233262, -0.35852225,  0.2458752 ],
       [-0.07380689, -0.20152667, -0.32220206,  0.30920178, -0.51717961,
         0.31233262,  1.        , -0.30980794,  0.03427709],
       [-0.24736541, -0.70220303,  0.05188949, -0.72716904,  0

In [37]:
# covariance matrix irrelevant for now
np.cov(img, rowvar = False, bias = True)

array([[5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.],
       [5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.],
       [5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.],
       [5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.],
       [5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.],
       [5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.],
       [5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.],
       [5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.]])

In [38]:
def correlation_mtx(img):
    return np.corrcoef(img, rowvar = False)

In [39]:
def correlation_val(img):
    # from paper
    res = 0
    mean_y = np.mean(img,axis=0)
    mean_x = np.mean(img,axis=1)
    var_y = np.var(img,axis=0)
    var_x = np.var(img,axis=1)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            res += (img[i,j] * (mean_x[i]*mean_y[j]) / (var_x[i]*var_y[j]))
    return res

In [40]:
def homogeneity(img):
    res = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            res += img[i,j] / (1 + abs(j - i))
    print(res)

In [41]:
def entropy(img):
    res = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            res += (img[i,j]+0.01) * math.log(img[i,j]+0.01) # If pixel value is 0 log is infinite
            #  add
    return res * -1

In [42]:
def variance(img):
    return np.var(img)

In [43]:
def contrast(img):
    res = 0
    res1 = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            n = math.pow(i-j,2)
            res += img[j,i] * n
    return res

In [44]:
# Pyradiomics glrlm
def longRunEmphasis(img):
    return 1

In [45]:
def runePercentage(img):
    return 1

In [46]:
def var_mtx(img):
    return np.var(img,axis=0), np.var(img,axis=1)

In [48]:
import pandas as pd

data = pd.DataFrame(columns=[
    'target',
    'correlation',
    'homogeneity',
    'entropy',
    'variance',
    'contrast',
    'longRunEmphasis',
    'runePercentage'
])

target = 'normal'
# corr = correlation_mtx(img)
corr = correlation_val(img)
homo = homogeneity(img)
ent = entropy(img)
var = variance(img)
cont = contrast(img)
lre = longRunEmphasis(img)
rp = runePercentage(img)

row = {
    'target': target,
    'correlation': corr,
    'homogeneity': homo,
    'entropy': ent,
    'variance': var,
    'contrast': cont,
    'longRunEmphasis': lre,
    'runePercentage': rp
    }
data.append(row,ignore_index=True)

3140.1


,target,correlation,homogeneity,entropy,variance,contrast,longRunEmphasis,runePercentage
0,normal,5.104328,None,-40626.159692,5460.0,84672.0,1,1
